# Intermediate Purchase Product Event Models

In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
PROJ_ROOT = Path().resolve().parents[3]
env_file_dir = PROJ_ROOT / '.env'
_ = load_dotenv(env_file_dir, verbose=True)

## About

Develop an **intermediate** model that gets events from sessions that

1. end in a purchase
2. do not end in a purchase

Both models only contain the events in which the product ID is present, so they exclude the events related to the checkout process (`event_type="checkout"` or `event_type="package_shipped"`).

### Use-Case

Other intermediate models that need to separate purchases from non-purchases will use these intermediate models.

### Notes

1. This notebook supports <kbd>Run</kbd> > <kbd>Run All Cells</kbd>.

## User Inputs

In [ ]:
#

In [ ]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        schema=os.getenv("UPLIMIT_SNOWFLAKE_SCHEMA"),
    )
)

## Connect

Load Jupyter SQL extension

In [ ]:
%load_ext sql

Connect to DuckDB database

In [ ]:
%sql engine --alias connection

## Models

### `int_product_non_purchases_filtered`

In [ ]:
%%sql
/* get events for sessions that did not end in a purchase */
WITH products_non_purchase_sessions AS (
    SELECT event_id,
           user_id,
           session_id,
           created_at,
           event_type,
           product_id,
           0 AS is_purchased
    FROM stg_postgres_events
    -- get sessions in which the last event does not indicate a purchase
    QUALIFY (
        LAST_VALUE(event_type)
        OVER(PARTITION BY session_id ORDER BY session_id, created_at)
    ) IN ('page_view', 'add_to_cart')
)
SELECT *
FROM products_non_purchase_sessions

### `int_product_purchases_filtered`

In [ ]:
%%sql
/* get the session ID for sessions ending in a purchase */
WITH sessions_with_purchase AS (
    SELECT DISTINCT(session_id) AS session_id
    FROM stg_postgres_events
    WHERE event_type IN ('checkout', 'package_shipped')
),
-- sessions_with_purchase AS (
--     SELECT DISTINCT(session_id) AS session_id
--     FROM stg_postgres_events
--     -- get sessions in which the last event indicates a purchase
--     QUALIFY (
--         LAST_VALUE(event_type)
--         OVER(PARTITION BY session_id ORDER BY session_id, created_at)
--     ) IN ('checkout', 'package_shipped')
-- ),
/* get events for sessions that did end in (convert to) a purchase */
products_purchase_sessions AS (
    SELECT s.event_id,
           s.user_id,
           s.session_id,
           s.created_at,
           s.event_type,
           s.product_id,
           1 AS is_purchased
    FROM stg_postgres_events s
    -- user INNER JOIN to only get sessions ending in a purchase
    INNER JOIN sessions_with_purchase sp USING (session_id)
)
SELECT *
FROM products_purchase_sessions

## Disconnect

Close connection

In [ ]:
%sql --close connection